In [1]:
# !pip install --upgrade langchain accelerate transformers
# !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!export PYTORCH_ENABLE_MPS_FALLBACK=1

In [2]:
import torch
from langchain.chains import (
    LLMChain,
    SequentialChain
)
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline, AutoTokenizer

In [3]:
# Globals
repo_id = 'tiiuae/falcon-7b-instruct'

tokenizer = AutoTokenizer.from_pretrained(repo_id)

generate_text = pipeline(
    "text-generation",
    max_length=1000,
    model=repo_id,
    torch_dtype=torch.float16,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    model_kwargs={
        'pad_token_id': tokenizer.eos_token_id
    }
)

transformer_pipeline = HuggingFacePipeline(pipeline=generate_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt = PromptTemplate.from_template(
    "Suggest a company name to describe a company that makes {product}?"
)

chain = LLMChain(
    llm=transformer_pipeline,
    prompt=prompt
)

In [8]:
product = "Artistic and visually stunning hand bags."
result = chain.run(product)
print(result)


Artistic Handbags


---

## SequentialChain

In [6]:
# prompt template 1: translate to english
first_prompt = PromptTemplate.from_template(
    "Summarize the text enclosed by <>: <{review}>"
)

chain_one = LLMChain(
    llm=transformer_pipeline, prompt=first_prompt,
    output_key="summary"
)

second_prompt = PromptTemplate.from_template(
    "Is the sentiment of the text enclosed by <> Positive or Negative? <{summary}>"
)

chain_two = LLMChain(
    llm=transformer_pipeline, prompt=second_prompt,
    output_key="sentiment"
)

third_prompt = PromptTemplate.from_template(
    "Extract key-phrases from the text enclosed by <>: <{summary}>"
)

chain_three = LLMChain(
    llm=transformer_pipeline, prompt=third_prompt,
    output_key="keywords"
)


In [7]:
# overall_chain: input= review 
# and output= language, english_review,summary, sentiment
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["review"],
    output_variables=["summary", "sentiment", "keywords"],
    verbose=False
)

review = "Hotel was kept clean and the staff around the pools, restaurant and bar worked hard. We were disappointed with the evening entertainment as there wasn’t much. We found the food very bland particularly at dinner and ended up going out for some evening meals. We had a pool view which was nice initially but realised the first morning it meant you were woken very early by the cleaning machines around the pool. The bed dipped in the middle and our room wasn’t brushed out or mopped during the duration of our stay."

result = overall_chain(review)
print(result)

{'review': 'Hotel was kept clean and the staff around the pools, restaurant and bar worked hard. We were disappointed with the evening entertainment as there wasn’t much. We found the food very bland particularly at dinner and ended up going out for some evening meals. We had a pool view which was nice initially but realised the first morning it meant you were woken very early by the cleaning machines around the pool. The bed dipped in the middle and our room wasn’t brushed out or mopped during the duration of our stay.', 'summary': '\nThe text describes a hotel with clean rooms, hardworking staff, and disappointing evening entertainment. The food was bland, and the pool view was nice but disturbed by cleaning machines. The bed dipped in the middle, and the room was not properly cleaned during their stay.', 'sentiment': ' Negative', 'keywords': ''}
